# Data Science for Business - Exploring Flight Delays in NYC


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Case Description

### Flight Delays

Flight delays are a common problem in the airline industry. They can be caused by a variety of factors, such as weather, air traffic control, and mechanical issues. Delays can be frustrating for passengers and costly for airlines. In this case study, we will work with a dataset containing flight delay data for all flights departing NYC in 2013. Besides delays, the dataset also includes useful meta-data on airlines, airports, weather, and planes. [More...](https://cran.r-project.org/web/packages/nycflights13/nycflights13.pdf).

The dataset structure is illustrated in the following diagram:

![Dataset Structure](nycflights13.png)


## Load Data

Let's start by loading the `flights` table, taking a look at the first few rows, and displaying the dimensions of the table.

In [2]:
flights = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/3014cdf1ad22beee81fe27adb73dcb317c6f296f/Session_02/flights.csv')

In [ ]:
flights.head(10)

In [ ]:
flights.shape

We continue by loading the remaining tables and taking a look at the first few rows of each table.

In [ ]:
airlines = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/3014cdf1ad22beee81fe27adb73dcb317c6f296f/Session_02/airlines.csv')
airlines.head(10)

In [ ]:
airports = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/3014cdf1ad22beee81fe27adb73dcb317c6f296f/Session_02/airports.csv')
airports.head(10)

In [7]:
# Your task: Load the *planes* and *weather* tables from CSV files, store them a variables called weather and planes and take a look at the first few rows of each table.

## Prepare Data

Before we start exploring the data, we need to prepare it. This includes cleaning the data (e.g., filtering rows with missing values or outliers), creating new features, and merging tables.

### Filter out observations

We start by filtering the `flights` table to exclude flights with a `distance` or `air_time` smaller or equal to 0. 

In [ ]:
flights = flights[flights['distance'] > 0]
flights.shape

In [9]:
# Your task: Filter out all flights with an air time of 0

### Construct new variables

The way how times are represented in this dataset is quite strange (look at the values `dep_time` or `arr_time` and the separate `hour` and `minute` columns). We will use the timestamps in the `time_hour` column to extract new time variables (`month` and `weekday`) and then remove the column.

In [10]:
flights['time_hour'] = pd.to_datetime(flights['time_hour'], utc=True)
flights['month_name'] = flights['time_hour'].dt.month_name()
flights['weekday_name'] = flights['time_hour'].dt.day_name()
flights = flights.drop(columns=['time_hour'])

In [ ]:
flights.head()

### Join tables

Finally, we will merge some of the tables to create a single table that contains all the information we need for our visualizations. For this, we will use different types of SQL-style joins.

![Types of Joins](join-venn.png)

We primarily use inner joins for matching observations when keys are equal:

![Inner Join](join-inner.png)


Let's demonstrate the differences between an **inner join** and a **left join** with some examples.

In [ ]:
flights_and_airlines = flights.merge(airlines, on='carrier', how='inner')
flights_and_airlines.shape

In [ ]:
flights_and_airlines = flights.merge(airlines, on='carrier', how='left')
flights_and_airlines.shape

In [14]:
# Your task: Do the same with flights and planes.

Left join of `flights` and `airports` using `origin` (left table) and `faa` (right table) as keys.

In [ ]:
flights_and_origin_airports = flights.merge(airports, left_on='origin', right_on='faa', how='left')
flights_and_origin_airports.shape

Left join of `flights` and `airports` using `dest` (left table) and `faa` (right table) as keys.

In [ ]:
flights_and_dest_airports = flights.merge(airports, left_on='dest', right_on='faa', how='left')
flights_and_dest_airports.shape

Left join of `flights` and `weather` using a composite key consisting of `origin`, `month`, `day`, and `hour`.


In [ ]:
flights_and_weather = flights.merge(weather, on=['origin', 'month', 'day', 'hour'], how='left')
flights_and_weather.shape

We can now join all the tables (`flights`, `airlines`, `planes`, `airports`, `weather`) into one comprehensive table.

In [ ]:
flights_all = (
    flights
    .merge(airlines, on='carrier', how='left')
    .merge(planes, on='tailnum', how='left')
    .merge(airports, left_on='origin', right_on='faa', how='left', suffixes=('_origin', '_dest'))
    .merge(airports, left_on='dest', right_on='faa', how='left', suffixes=('', '_dest'))
    .merge(weather, on=['origin', 'month', 'day', 'hour'], how='left')
)

flights_all.head()


In [ ]:
flights_all.shape

In [ ]:
flights_all.columns

The joins created some duplicate columns. Let's remove them and renaming columns for clarity.

In [21]:
flights_all = flights_all.drop(columns=['year_y', 'time_hour'])

flights_all = flights_all.rename(columns={
    'year_x': 'year',
})

In [ ]:
flights_all.head()

In [ ]:
flights_all.columns

## Explore Data through Visualizations

We will now visualize categorical and numerical variables to gain insights into the data. We will use the Seaborn library for this purpose. Seaborn is a data visualization library based on Matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics. But it also offers a low-level interface for detailed customization of plots.


### Plots for Categorical Variables

We begin by plotting a bar chart of the number of flights by `origin` airport. The function `countplot` is used to show the counts of observations in each categorical bin using bars. Then we use the functions `title`, `xlabel`, and `ylabel` to set a title and labels of the x and y axes, respectively.


In [ ]:
sns.countplot(data=flights_all, x='origin')
plt.title('Number of Flights by Origin Airport')
plt.xlabel('Origin Airport')
plt.ylabel('Number of Flights')
plt.show()


In [25]:
# Your task: Do the same for the number of flights by airline.

To order the bars by frequency, we can use the `order` parameter of the `countplot` function.


In [ ]:
sns.countplot(data=flights_all, x='carrier', order=flights_all['carrier'].value_counts().index)
plt.title('Who are the biggest airlines in NYC?')
plt.xlabel('Airline Code')
plt.ylabel('Number of Flights Leaving NYC')
plt.xticks(rotation=90)
plt.show()


### Plots for Numerical Variables

Next, we explore the distribution of the `distance` of all flights using the `displot` function. Using the value *hist* for the `kind` parameter, we can plot a histogram. The *bins* parameter specifies the number of bins to use for the histogram.


In [ ]:
sns.displot(data=flights_all, kind='hist', x='distance', bins=30)
plt.title('Distribution of Flight Distances')
plt.xlabel('Distance (miles)')
plt.ylabel('Number of Flights')
plt.show()


In [28]:
# Your turn: Do the same for airtime. 

Using the value *kde* for the `kind` parameter, we can plot a kernel density estimate. The *fill* parameter specifies whether to fill the area under the KDE curve.

In [ ]:
sns.displot(data=flights_all, kind='kde', x='distance', fill=True)
plt.title('Distribution of Flight Distances')
plt.xlabel('Distance (miles)')
plt.ylabel('Number of Flights')
plt.show()


If you want to create the same plot for different subsets of the data, you can use the `FacetGrid` function. It maps a low-level function to create a plot (here: `histplot`) to each subset of the data defined by the level of a categorical variable. 

In [ ]:
g = sns.FacetGrid(flights_all, col="origin")
g.map(sns.histplot, "distance")
g.set_axis_labels("Distance (miles)", "Number of Flights")
plt.show()


Box plots provide another alternative for visualizing the distribution of numerical variables. The low-level function `boxplot` can be used to create a box plot for a numerical variable (here: `arr_delay`), optinally grouped by a categorical variable (here: `origin`).

In [ ]:
sns.boxplot(data=flights_all, y='distance', x='origin')
plt.title('Box Plot of Flight Distances by Origin')
plt.xlabel('Origin Airport')
plt.ylabel('Distance (miles)')
plt.show()


In [50]:
# Your turn: Do the same for airlines instead of origin airports.

The above plots show the relationship between a categorical and a numerical variable. We can also visualize the relationship between two numerical variables. The `relplot` function offers various options for this purpose. Here, we use the `scatterplot` kind to create a scatter plot of `distance` and `air_time`.

In [ ]:
sns.relplot(data=flights_all, kind='scatter', x='distance', y='air_time', alpha=0.5)
plt.title('Scatter Plot: Distance vs Air Time')
plt.xlabel('Distance (miles)')
plt.ylabel('Air Time (minutes)')
plt.show()


In [55]:
#Your task: Do the same for delay and a numerical weather variable of your choice.